In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import datetime
from functools import partial
import os

todo_items = []

# 공통 레이아웃
layout_full = widgets.Layout(width='100%')
layout_half = widgets.Layout(width='48%')
layout_button = widgets.Layout(width='100%', height='35px')
layout_label = widgets.Layout(width='100px')

# 전체 진행률 표시
overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, layout=widgets.Layout(width='50%', height='24px'), style={'bar_color': '#000'})
overall_progress_label = widgets.Label(value='(0%)')

# 할 일 박스
todo_list_box = widgets.VBox(layout=widgets.Layout(width='100%', padding='10px'))

# 전체 진행률 업데이트
def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"({int(avg)}%)"
    overall_progress_bar.style = {'bar_color': get_progress_color(avg)}

def get_progress_color(p):
    gray = int(p * 2.55)
    return f'rgb({gray},{gray},{gray})'

# 할 일 목록 갱신
def refresh_list():
    todo_list_box.children = []
    for idx, item in enumerate(todo_items, 1):
        title = widgets.Label(value=item['title'], layout=layout_full)
        progress = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=layout_full)
        progress.style = {'bar_color': get_progress_color(item['progress'])}
        slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_full)

        date_label = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else '-', layout=layout_full)
        priority_label = widgets.Label(value=item['priority'], layout=layout_full)

        complete_btn = widgets.Button(description="완료✅", layout=layout_button, button_style='')
        delete_btn = widgets.Button(description="삭제❌", layout=layout_button, button_style='')

        button_box = widgets.VBox([complete_btn, delete_btn], layout=layout_full)

        def on_progress_change(change, i=item, bar=progress):
            i['progress'] = change['new']
            bar.value = change['new']
            bar.style = {'bar_color': get_progress_color(change['new'])}
            update_overall_progress()

        def on_complete(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()

        def on_delete(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()

        slider.observe(on_progress_change, names='value')
        complete_btn.on_click(partial(on_complete, i=item))
        delete_btn.on_click(partial(on_delete, i=item))

        box = widgets.VBox([
            widgets.HTML(f"<b># {idx}</b>"),
            widgets.HBox([widgets.Label("할 일", layout=layout_label), title]),
            widgets.HBox([widgets.Label("진행률", layout=layout_label), progress]),
            widgets.HBox([widgets.Label("입력", layout=layout_label), slider]),
            widgets.HBox([widgets.Label("마감일", layout=layout_label), date_label]),
            widgets.HBox([widgets.Label("중요도", layout=layout_label), priority_label]),
            widgets.HBox([widgets.Label("작업", layout=layout_label), button_box]),
        ], layout=widgets.Layout(border='1px solid #aaa', padding='10px', margin='5px 0'))
        todo_list_box.children += (box,)

# 입력 영역
title_input = widgets.Text(placeholder='할 일 입력', layout=layout_full)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_full)
due_input = widgets.DatePicker(layout=layout_full)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_full)
add_btn = widgets.Button(description="추가", button_style='success', layout=widgets.Layout(width='100%', height='40px'))

def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        update_overall_progress()
        refresh_list()

add_btn.on_click(add_task)

# 저장 / 불러오기
def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([
            {
                **i,
                'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None
            } for i in todo_items
        ], f)

def load_data(b):
    if os.path.exists('todo_data.json'):
        with open('todo_data.json', 'r') as f:
            raw = json.load(f)
            todo_items.clear()
            for i in raw:
                todo_items.append({
                    'title': i['title'],
                    'progress': i['progress'],
                    'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                    'priority': i.get('priority', '보통')
                })
        update_overall_progress()
        refresh_list()

save_btn = widgets.Button(description='저장', button_style='info', layout=layout_half)
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=layout_half)
save_btn.on_click(save_data)
load_btn.on_click(load_data)

# 전체 출력
display(widgets.VBox([
    widgets.HTML("<h2>📋 DAVID mk25</h2>"),
    widgets.HBox([widgets.Label("할 일", layout=layout_label), title_input]),
    widgets.HBox([widgets.Label("진행률", layout=layout_label), progress_input]),
    widgets.HBox([widgets.Label("마감일", layout=layout_label), due_input]),
    widgets.HBox([widgets.Label("중요도", layout=layout_label), priority_input]),
    widgets.HBox([add_btn]),
    widgets.HBox([save_btn, load_btn]),
    widgets.HBox([overall_progress_bar, overall_progress_label]),
    todo_list_box
], layout=widgets.Layout(width='100%')))

update_overall_progress()
refresh_list()
